In [31]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [966 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 1,196 kB in 1s (808 kB/s)
Reading package lists... Done


In [32]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, to_date
import time
from datetime import datetime

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [33]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [34]:
# add year column based off 'date' column.
df_with_year = df.withColumn("year", year(df['date']))
df_with_year.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [45]:
# 2. Create a temporary view of the DataFrame.
df_with_year.createOrReplaceTempView('home_sales')


In [46]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT year, float(avg(price)) FROM homes WHERE bedrooms=4 GROUP BY year").show()


+----+----------+
|year|avg(price)|
+----+----------+
|2022| 296363.88|
|2019|  300263.7|
|2020| 298353.78|
|2021| 301819.44|
+----+----------+



In [47]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("SELECT date_built, float(avg(price)) FROM homes WHERE bedrooms=3 AND bathrooms=3 GROUP BY date_built").show()


+----------+----------+
|date_built|avg(price)|
+----------+----------+
|      2015| 288770.28|
|      2013| 295962.28|
|      2014| 290852.28|
|      2012|  293683.2|
|      2016| 290555.06|
|      2010| 292859.62|
|      2011| 291117.47|
|      2017| 292676.78|
+----------+----------+



In [48]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("SELECT date_built, float(avg(price)) \
  FROM homes \
  WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living >= 2000\
  GROUP BY date_built").show()


+----------+----------+
|date_built|avg(price)|
+----------+----------+
|      2015| 297609.97|
|      2013| 303676.78|
|      2014| 298264.72|
|      2012| 307539.97|
|      2016|  293965.1|
|      2010| 285010.22|
|      2011|  276553.8|
|      2017| 280317.56|
+----------+----------+



In [49]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
spark.sql("SELECT float(avg(price)), avg(view) \
  FROM homes \
  WHERE price >= 350000 \
  GROUP BY price").show()


start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----------+---------+
|avg(price)|avg(view)|
+----------+---------+
|  430666.0|     21.0|
|  374948.0|     23.5|
|  363904.0|     25.5|
|  351086.0|     33.0|
|  978727.0|     83.5|
|  359796.0|     15.0|
|  444773.0|     11.0|
| 1456592.0|     89.0|
|  398172.0|     50.0|
|  356149.0|      1.0|
|  428490.0|     23.0|
|  419385.0|     18.0|
|  443881.0|     34.0|
|  392791.0|     36.0|
|  380033.0|     38.0|
|  400301.0|     30.0|
|  393437.0|     21.0|
|  443032.0|     14.0|
|  419000.0|     33.0|
|  395745.0|     25.0|
+----------+---------+
only showing top 20 rows

--- 7.033348083496094e-05 seconds ---


In [50]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [51]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [52]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
spark.sql("SELECT float(avg(price)), avg(view) \
  FROM homes \
  WHERE price >= 350000 \
  GROUP BY price").show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


+----------+---------+
|avg(price)|avg(view)|
+----------+---------+
|  430666.0|     21.0|
|  374948.0|     23.5|
|  363904.0|     25.5|
|  351086.0|     33.0|
|  978727.0|     83.5|
|  359796.0|     15.0|
|  444773.0|     11.0|
| 1456592.0|     89.0|
|  398172.0|     50.0|
|  356149.0|      1.0|
|  428490.0|     23.0|
|  419385.0|     18.0|
|  443881.0|     34.0|
|  392791.0|     36.0|
|  380033.0|     38.0|
|  400301.0|     30.0|
|  393437.0|     21.0|
|  443032.0|     14.0|
|  419000.0|     33.0|
|  395745.0|     25.0|
+----------+---------+
only showing top 20 rows

--- 9.608268737792969e-05 seconds ---


In [53]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [54]:
# 11. Read the parquet formatted data.
p_df_with_year = spark.read.parquet("home_sales_partitioned")

In [55]:
# 12. Create a temporary table for the parquet data.
p_df_with_year.createOrReplaceTempView('p_df_with_year')

In [56]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

spark.sql("SELECT float(avg(price)), avg(view) \
  FROM p_df_with_year \
  WHERE price >= 350000 \
  GROUP BY price").show()
start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----------+---------+
|avg(price)|avg(view)|
+----------+---------+
| 1456592.0|     89.0|
|  399735.0|     14.0|
|  446974.0|     30.0|
|  410754.0|      8.0|
| 1287887.0|     91.0|
|  353501.0|     25.0|
|  396523.0|     31.5|
|  351086.0|     33.0|
|  356149.0|      1.0|
|  380033.0|     38.0|
|  376270.0|     21.0|
|  422491.0|     13.0|
|  431136.0|     10.0|
|  441884.0|     16.0|
|  608011.0|     69.0|
|  431379.0|     35.0|
|  363904.0|     25.5|
|  444773.0|     11.0|
|  419385.0|     18.0|
|  393437.0|     21.0|
+----------+---------+
only showing top 20 rows

--- 7.271766662597656e-05 seconds ---


In [57]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [60]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is till cached.")
else:
  print("the table has been uncached.")


the table has been uncached.
